In [35]:
!pip install tensorflow
!pip install scikit-learn


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [36]:
!pip install pandas
!pip install numpy
!pip install matplotlib
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as pl
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [37]:
iris=load_iris()

In [38]:
iris_x,iris_y=iris.data[:-1,:],iris.target[:-1]
iris_y=pd.get_dummies(iris_y).values#is used to encode the labels in the one hot encoded forms as vectors
train_x,test_x,train_y,test_y=train_test_split(iris_x,iris_y,test_size=0.33,random_state=42)

In [39]:
numfeatures=train_x.shape[1]
numLables=train_y.shape[1]
print(numfeatures)
print(numLables)

4
3


In [40]:
train_x=tf.constant(train_x,dtype="float32")
train_y=tf.constant(train_y,dtype="float32")
test_x=tf.constant(test_x,dtype="float32")
test_y=tf.constant(test_y,dtype="float32")

In [41]:
w=tf.Variable(tf.zeros([4,3]))
b=tf.Variable(tf.zeros([3]))

In [42]:
weights=tf.Variable(tf.random.normal([numfeatures,numLables],mean=0,stddev=0.01,name="weights"),dtype="float32")
bias=tf.Variable(tf.random.normal([1,numLables],mean=0,stddev=0.01,name="bias"))


In [43]:
def logistic_regression(x):
    apply_weights=tf.matmul(x,weights,name="apply_weights")
    apply_bias=tf.add(apply_weights,bias,name="apply_bias")
    sigmoid=tf.nn.sigmoid(apply_bias,name="activation_function")
    return sigmoid

In [44]:
# Number of epochs
epochno = 700

# Learning rate schedule
learningrate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0008,
    decay_steps=train_x.shape[0],
    decay_rate=0.95,
    staircase=True
)

# Use cross-entropy loss for classification
loss_object = tf.keras.losses.CategoricalCrossentropy()

# Optimizer
optimizer = tf.keras.optimizers.SGD(learningrate)

# Accuracy function
def accuracy(y_pred, y_true):
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    return tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Optimization step
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = loss_object(y, pred)   # order is (y_true, y_pred)
    gradients = g.gradient(loss, [weights, bias])
    optimizer.apply_gradients(zip(gradients, [weights, bias]))
    return loss

# Training loop
display_step = 10
epoch_values, accuracy_values, loss_values = [], [], []
loss, diff = 0, 1

for i in range(epochno):
    if i > 1 and diff < 0.0001:
        print("change in loss %g; convergence." % diff)
        break
    else:
        # Run training step
        run_optimization(train_x, train_y)

        if i % display_step == 0:
            epoch_values.append(i)

            pred = logistic_regression(test_x)
            newLoss = loss_object(test_y, pred)
            loss_values.append(newLoss)

            acc = accuracy(pred, test_y)
            accuracy_values.append(acc)

            diff = abs(newLoss - loss)
            loss = newLoss

            print("step %d, test accuracy %g, loss %g, change in loss %g" %
                  (i, acc, newLoss, diff))

print("final accuracy on test set: %s" % acc.numpy())

step 0, test accuracy 0.32, loss 1.09042, change in loss 1.09042
step 10, test accuracy 0.32, loss 1.08824, change in loss 0.00217986
step 20, test accuracy 0.32, loss 1.08613, change in loss 0.00211012
step 30, test accuracy 0.32, loss 1.08409, change in loss 0.00204659
step 40, test accuracy 0.32, loss 1.0821, change in loss 0.00198889
step 50, test accuracy 0.32, loss 1.08016, change in loss 0.00193608
step 60, test accuracy 0.32, loss 1.07827, change in loss 0.00188839
step 70, test accuracy 0.32, loss 1.07643, change in loss 0.00184488
step 80, test accuracy 0.32, loss 1.07462, change in loss 0.00180531
step 90, test accuracy 0.32, loss 1.07285, change in loss 0.00176954
step 100, test accuracy 0.32, loss 1.07113, change in loss 0.00172019
step 110, test accuracy 0.32, loss 1.06951, change in loss 0.00162363
step 120, test accuracy 0.32, loss 1.06791, change in loss 0.00160015
step 130, test accuracy 0.32, loss 1.06633, change in loss 0.00157869
step 140, test accuracy 0.32, loss 